# 主要是測試各種功能用的，自行增加區塊來測試(記得加註解)

## torch測試及可用顯存測試

In [2]:
import torch
from psutil import virtual_memory
print(torch.cuda.is_available())
GPUmem = torch.cuda.mem_get_info()
mem = GPUmem[1] /1024 / 1024 / 1024
mem += virtual_memory().total /1024 / 1024 / 1024 / 2
mem

True


27.873252868652344

## 功能檔案分離測試

In [ ]:
import WebDisplay
data = {
    'name': 'example',
    'value': 42,
    'status': 'active'
}
web_display = WebDisplay.WebDisplay(data)
web_display.print_data()

## 下載YouTube影片及聲音測試

In [1]:
from bs4 import BeautifulSoup
from pytubefix import YouTube
yt = YouTube('https://music.youtube.com/watch?v=NycFr6D6DSw&si=przy_9oxIVQs5HI9')
print('download...')
yt.streams.get_audio_only().download(output_path='download/')   #下載音訊
print('ok!')

download...
ok!


In [ ]:
acc = yt.streams.get_audio_only()
acc.download(output_path='download/')   #下載音訊

### 加入手動輸入連結

In [ ]:
url = input()

In [ ]:
from YouTubeDownload import YouTubeDownload
from MusicSeparation import MusicSeparation
import os
vedio = YouTubeDownload.download_youtube_video(url)
audio = YouTubeDownload.download_youtube_audio(url)
separation_audio = MusicSeparation.run_separation(audio)
merge = YouTubeDownload.merge_video_audio(vedio, separation_audio[0])
print(vedio, audio , merge, separation_audio)
#os.system(merge)
separation_audio

In [ ]:
os.system(merge)
audio

In [ ]:
#測試用，記得刪
from MusicSeparation import MusicSeparation
from YouTubeDownload import YouTubeDownload
separation_audio = ['h:\\NFU\\score\\score/output\\光輝歲月 - Beyond ( 黃家駒, 黃貫中, 黃家強, 葉世榮 )[ lyrics ]\\光輝歲月 - Beyond ( 黃家駒, 黃貫中, 黃家強, 葉世榮 )[ lyrics ]-other.wav',
 'h:\\NFU\\score\\score/output\\光輝歲月 - Beyond ( 黃家駒, 黃貫中, 黃家強, 葉世榮 )[ lyrics ]\\光輝歲月 - Beyond ( 黃家駒, 黃貫中, 黃家強, 葉世榮 )[ lyrics ]-vocals.wav']

### 整合ASR辨識

In [ ]:
ASR_result = MusicSeparation.run_ASR(separation_audio[1])
YouTubeDownload.download_captions_by_language_code(video_url=url,language_code=ASR_result[1])

In [ ]:
from YouTubeDownload import YouTubeDownload
YouTubeDownload.download_captions_by_language_code(video_url=url,language_code='ja')

## NeMo測試(確認無用)

In [ ]:
import nemo
import nemo.collections.asr as nemo_asr

print(nemo.__version__)


## Whisper字幕辨識測試

In [ ]:
import whisper
whisper.available_models()

In [ ]:
import os
import torch
import tkinter as tk
from tkinter import filedialog
import whisper

#建立Tk主視窗，但隱藏
root = tk.Tk()
root.withdraw()
root.attributes('-topmost', True)  #對話框出現在最前方

#選擇音訊檔案
current_dir = os.getcwd()
file_path = filedialog.askopenfilename(
    title="請選擇音訊檔案",
    initialdir=current_dir,
    filetypes=[("音訊檔案", "*.wav *.mp3 *.m4a *.flac"), ("所有檔案", "*.*")]
)

if not file_path:
    print("未選擇檔案")
    exit()

print("選擇的檔案:", file_path)

#載入Whisper模型
if(torch.cuda.is_available()):
    model = whisper.load_model("large-v3", device="cuda")
else:
    model = whisper.load_model("large-v3", device="cpu")

#轉錄音訊，Whisper會自動偵測語言
result = model.transcribe(file_path)
print("偵測語言:", result["language"])

#輸出資料夾:與音訊檔同一層
audio_dir = os.path.dirname(file_path)

#取得音訊檔名稱(不含副檔名)
filename = os.path.splitext(os.path.basename(file_path))[0]

#---SRT---
with open(os.path.join(audio_dir, f"{filename}.srt"), "w", encoding="utf-8") as f:
    for i, seg in enumerate(result["segments"], start=1):
        text = seg['text'].strip()
        if not text:
            continue
        f.write(f"{i}\n")
        start = seg["start"]
        end = seg["end"]
        start_time = f"{int(start//3600):02}:{int((start%3600)//60):02}:{int(start%60):02},{int((start%1)*1000):03}"
        end_time = f"{int(end//3600):02}:{int((end%3600)//60):02}:{int(end%60):02},{int((end%1)*1000):03}"
        f.write(f"{start_time} --> {end_time}\n")
        f.write(f"{text}\n\n")

#---VTT---
with open(os.path.join(audio_dir, f"{filename}.vtt"), "w", encoding="utf-8") as f:
    f.write("WEBVTT\n\n")
    for seg in result["segments"]:
        text = seg['text'].strip()
        if not text:
            continue
        start = seg["start"]
        end = seg["end"]
        start_time = f"{int(start//3600):02}:{int((start%3600)//60):02}:{int(start%60):02}.{int((start%1)*1000):03}"
        end_time = f"{int(end//3600):02}:{int((end%3600)//60):02}:{int(end%60):02}.{int((end%1)*1000):03}"
        f.write(f"{start_time} --> {end_time}\n")
        f.write(f"{text}\n\n")

#---TXT---
with open(os.path.join(audio_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
    for seg in result["segments"]:
        text = seg['text'].strip()
        if not text:
            continue
        start = seg["start"]
        end = seg["end"]
        start_time = f"{int(start//3600):02}:{int((start%3600)//60):02}:{int(start%60):02}.{int((start%1)*1000):03}"
        end_time = f"{int(end//3600):02}:{int((end%3600)//60):02}:{int(end%60):02}.{int((end%1)*1000):03}"
        f.write(f"[{start_time} --> {end_time}] {text}\n")

print(f"已輸出至資料夾:{audio_dir}")


### 分離功能至MusicSeparation.py內測試

In [ ]:
from MusicSeparation import MusicSeparation
#MusicSeparation.run_ASR(r"H:/NFU/score/score/output/Immortals/Immortals-vocals.wav")    #若無傳入值則由使用者手動選取
MusicSeparation.run_ASR()

## Whisper微調測試(官方範例)

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

#載入文件範例("mozilla-foundation/common_voice_11_0"是線上的資料，要自己改)
common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "hi", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "hi", split="test", use_auth_token=True)

print(common_voice)

In [ ]:
from transformers import WhisperFeatureExtractor

#特徵提取器(預處理音檔)_選擇模型
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [ ]:
from transformers import WhisperTokenizer

#模型輸出後處理為文字格式的分詞器
#language要自己改
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Hindi", task="transcribe")

In [ ]:
input_str = common_voice["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Hindi", task="transcribe")

print(common_voice["train"][0])
"""輸出格式範例(採樣率48K)
{'audio': {'path': '/home/sanchit_huggingface_co/.cache/huggingface/datasets/downloads/extracted/607848c7e74a89a3b5225c0fa5ffb9470e39b7f11112db614962076a847f3abf/cv-corpus-11.0-2022-09-21/hi/clips/common_voice_hi_25998259.mp3', 
           'array': array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 9.6724887e-07,
       1.5334779e-06, 1.0415988e-06], dtype=float32), 
           'sampling_rate': 48000},
 'sentence': 'खीर की मिठास पर गरमाई बिहार की सियासत, कुशवाहा ने दी सफाई'}
"""

In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

print(common_voice["train"][0])
"""輸出格式範例(下採樣至16K)
{'audio': {'path': '/home/sanchit_huggingface_co/.cache/huggingface/datasets/downloads/extracted/607848c7e74a89a3b5225c0fa5ffb9470e39b7f11112db614962076a847f3abf/cv-corpus-11.0-2022-09-21/hi/clips/common_voice_hi_25998259.mp3', 
           'array': array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
       -3.4206650e-07,  3.2979898e-07,  1.0042874e-06], dtype=float32),
           'sampling_rate': 16000},
 'sentence': 'खीर की मिठास पर गरमाई बिहार की सियासत, कुशवाहा ने दी सफाई'}
"""

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    #(翻譯)載入音檔取樣率改至16K
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    #(翻譯)從輸入音訊數組計算 log-Mel 輸入特徵
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    #(翻譯)將目標文字編碼為標籤ID
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

#用.map函式在所有訓練樣本應用上方函式
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=4)
"""
注意：目前 datasets 使用 torchaudio 和 librosa 進行音訊載入與重取樣。如果您想實作自訂的資料載入/重取樣，可以使用 "path" 欄位取得音訊檔案路徑，並忽略 "audio" 欄位。
"""

In [ ]:
#定義數據收集器
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
#初始化數據整理器
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
)

In [ ]:
import evaluate

#wer代表詞錯誤率
metric = evaluate.load("wer")

#錯誤率評估函式
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
#載入預訓練checkpoint
from transformers import WhisperForConditionalGeneration

#模型自己改
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [ ]:
from transformers import Seq2SeqTrainingArguments

#如果不想將模型checkpoint上傳到Hub，設定push_to_hub=False
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-hi",  # change to a repo name of your choice
    per_device_train_batch_size=16, #如果出現OOM(顯存超載)錯誤將此減半
    gradient_accumulation_steps=1,  #並將此乘2
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [ ]:
#訓練模型
trainer.train()

In [ ]:
#設定上傳模型數值
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0", #標籤，記得改
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset (翻譯)給資料集設定一個你喜歡的名子
    "dataset_args": "config: hi, split: test",
    "language": "hi", #語言自己改
    "model_name": "Whisper Small Hi - Sanchit Gandhi",  # a 'pretty' name for your model (翻譯)給模型一個好名子
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

In [ ]:
#上傳模型
trainer.push_to_hub(**kwargs)

### 訓練完後如何使用

In [ ]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="sanchit-gandhi/whisper-small-hi") #改成你自己的模型tag

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(sources=["microphone"], type="filepath"),
    outputs="text",
    title="Whisper Small Hindi", #標題自訂
    description="Realtime demo for Hindi speech recognition using a fine-tuned Whisper small model.",
)
    
iface.launch()